In [ ]:
pip install aif360 pandas scikit-learn


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.postprocessing import RejectOptionClassification
from aif360.metrics import BinaryLabelDatasetMetric


In [ ]:
# Load the dataset (Example: UCI Adult dataset)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]
df = pd.read_csv(url, names=columns, header=None, na_values="?", skipinitialspace=True)

# Preprocess the dataset
df = df.dropna()
df['income'] = df['income'].apply(lambda x: 1 if x == ">50K" else 0)

print("Sample Data:\n", df.head())


In [ ]:
# Define sensitive attributes and label
privileged_groups = [{'sex': 1}]  # Male
unprivileged_groups = [{'sex': 0}]  # Female

# Convert to AIF360 dataset
dataset = StandardDataset(df, label_name='income', favorable_classes=[1], 
                          protected_attribute_names=['sex'], privileged_classes=[[1]])

# Evaluate fairness
metric = BinaryLabelDatasetMetric(dataset, privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)
print("Disparate Impact:", metric.disparate_impact())


In [ ]:
# Apply reweighing
rw = Reweighing(privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)
reweighted_dataset = rw.fit_transform(dataset)

# Train a classifier on the reweighted dataset
X = reweighted_dataset.features
y = reweighted_dataset.labels.ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))


In [ ]:
# Post-processing with Reject Option Classification
roc = RejectOptionClassification(unprivileged_groups=privileged_groups, privileged_groups=unprivileged_groups)
roc = roc.fit(reweighted_dataset, reweighted_dataset)
predicted_dataset = roc.predict(reweighted_dataset)

# Evaluate fairness post-processing
post_metric = BinaryLabelDatasetMetric(predicted_dataset, privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)
print("Disparate Impact After Mitigation:", post_metric.disparate_impact())
